Uvoz knjižnic in branje podatkov iz datotek:

In [2]:
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import numpy as np

In [3]:
list_of_cvs = ["./podatki/kd{}.csv".format(i) for i in range(2010,2020)]
frames = [pd.read_csv(csv,sep=';', encoding="UTF-8", dtype={'MesecStoritve': str}) for csv in list_of_cvs]
df = pd.concat(frames)

C:\Users\Uporabnik\AppData\Local\Temp\ipykernel_2284\1644075804.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(csv,sep=';', encoding="UTF-8", dtype={'MesecStoritve': str}) for csv in list_of_cvs]
C:\Users\Uporabnik\AppData\Local\Temp\ipykernel_2284\1644075804.py:2: DtypeWarning: Columns (14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(csv,sep=';', encoding="UTF-8", dtype={'MesecStoritve': str}) for csv in list_of_cvs]


In [57]:
#group by mesec, count unique id
kd_leto = df.groupby(['MesecStoritve', 'UraStoritve','ZaporednaStevilkaKD'])
leto_counter = defaultdict(int)

for name,v in kd_leto:
    index = int((name[0].split(".")[1]))
    if index >= 2010 and index < 2020:
        leto_counter[index] +=1
    


list_mes_counter = sorted(leto_counter.items())
mesec, counter = zip(*list_mes_counter)
""" plt.figure(figsize=(15,10))
plt.plot(mesec,counter)
plt.show() """

' plt.figure(figsize=(15,10))\nplt.plot(mesec,counter)\nplt.show() '

In [5]:
pijaniMladoletniki = df.groupby(['StarostniRazred', 'Skoda'])
skodaStarostStevilo = defaultdict(lambda: defaultdict(int))
for name, v in pijaniMladoletniki:
    skodaStarostStevilo[name[1]][name[0]] = len(v['ZaporednaStevilkaKD'].unique())

skodaStarostStevilo

defaultdict(<function __main__.<lambda>()>,
            {'1.000 - 10.000 EUR': defaultdict(int,
                         {'00-07': 130,
                          '07-14': 245,
                          '14-16': 172,
                          '16-18': 327,
                          '18-24': 2718,
                          '24-34': 11476,
                          '34-44': 15831,
                          '44-54': 15437,
                          '54-64': 11420,
                          'NAD 64': 8733,
                          'NI PODATKA': 45,
                          'NI POJAVA': 7,
                          'PRAVNA OSEBA': 34901}),
             '10.000 - 100.000 EUR': defaultdict(int,
                         {'00-07': 4,
                          '07-14': 26,
                          '14-16': 17,
                          '16-18': 18,
                          '18-24': 247,
                          '24-34': 1688,
                          '34-44': 2729,
                         

In [18]:
def wihtout_keys(d, keys):
    return {x: d[x] for x in d if x not in keys}

In [55]:
## Največji kršitelji po rangu vrednosti v procentih. BREZ PRAVNIH OSEB
import operator
invalid = {"PRAVNA OSEBA"}
for k, v in skodaStarostStevilo.items():
    v = wihtout_keys(v, invalid)
    print("Vrednost: %s \n Starost: %s\n Delež:%.2f %%" % (k, max(v.items(), key=operator.itemgetter(1)), max(v.values())/sum(v.values())*100))

Vrednost: 1.000 - 10.000 EUR 
 Starost: ('34-44', 15831)
 Delež:23.79 %
Vrednost: 10.000 - 100.000 EUR 
 Starost: ('44-54', 2756)
 Delež:24.77 %
Vrednost: 100 - 1.000 EUR 
 Starost: ('34-44', 37573)
 Delež:19.84 %
Vrednost: 100.000 - 500.000 EUR 
 Starost: ('44-54', 245)
 Delež:26.98 %
Vrednost: BREZ 
 Starost: ('34-44', 77155)
 Delež:20.64 %
Vrednost: DO 100 EUR 
 Starost: ('34-44', 18599)
 Delež:19.85 %
Vrednost: NAD 500.000 EUR 
 Starost: ('44-54', 41)
 Delež:28.47 %


In [79]:
#Dictionary ENOTA{kaznivo dejanje: stevilo teh dejanj}
#idea Porazdelitev vrst kaznivih dejanj po posamezni enoti/občini/karkol se jim reče

#ZaporednaStevilkaKD;MesecStoritve;UraStoritve;OpisKD

kd_leto = df.groupby(['MesecStoritve', 'UraStoritve','ZaporednaStevilkaKD', 'OpisKD', 'PUStoritveKD'])

kraj_dejanje = defaultdict(lambda: defaultdict(int))
for k, v in kd_leto:
    #print(k, len(v))
    kraj_dejanje[k[-1]][k[-2].split(" - ")[1]] += len(v)

('01.1944', '00:00-00:59', 15723, 'KZ08/101-/1// - HUDODELSTVA ZOPER ÈLOVEÈNOST', 'PU KRANJ') 1
('01.1978', '00:00-00:59', 75804, 'KZ08/173-/1// - SPOLNI NAPAD NA OSEBO,  MLAJŠO OD PETNAJST LET', 'PU KRANJ') 2
('01.1979', '00:00-00:59', 45042, 'KZ12/118*/1// - POVZROÈITEV SMRTI IZ MALOMARNOSTI', 'PU MURSKA SOBOTA') 2
('01.1981', '00:00-00:59', 42610, 'KZ08/173-/3// - SPOLNI NAPAD NA OSEBO,  MLAJŠO OD PETNAJST LET', 'PU NOVO MESTO') 1
('01.1982', '00:00-00:59', 22407, 'KZ08/211-/1/S/ - GOLJUFIJA', 'PU LJUBLJANA') 2
('01.1982', '00:00-00:59', 24932, 'KD02/103/1// - SPOLNI NAPAD NA OSEBO MLAJŠO OD 14 LET', 'PU LJUBLJANA') 2
('01.1983', '10:00-10:59', 3308, 'KD02/103/1// - SPOLNI NAPAD NA OSEBO MLAJŠO OD 14 LET', 'PU KOPER') 2
('01.1984', '12:00-12:59', 9552, 'KZ12/173*/3// - SPOLNI NAPAD NA OSEBO,  MLAJŠO OD PETNAJST LET', 'PU NOVO MESTO') 2
('01.1984', '23:00-23:59', 40075, 'KZ08/173-/1// - SPOLNI NAPAD NA OSEBO,  MLAJŠO OD PETNAJST LET', 'PU KOPER') 2

In [88]:
for k, v in kraj_dejanje.items():
    print(v)

defaultdict(<class 'int'>, {'HUDODELSTVA ZOPER ÈLOVEÈNOST': 1, 'SPOLNI NAPAD NA OSEBO,  MLAJŠO OD PETNAJST LET': 262, 'NASILJE V DRUŽINI': 1719, 'POSILSTVO': 97, 'GOLJUFIJA': 6483, 'ZLORABA POLOŽAJA ALI ZAUPANJA PRI GOSPODARSKI DEJAVNOSTI': 1348, 'POSLOVNA GOLJUFIJA': 5371, 'PONAREJANJE LISTIN': 1884, 'NEPLAÈEVANJE PREŽIVNINE': 6576, 'PONAREDITEV ALI UNIÈENJE POSLOVNIH LISTIN': 991, 'IZNEVERJENJE': 575, 'POSEBNI PRIMERI PONAREJANJA LISTIN': 142, 'ZANEMARJANJE MLADOLETNE OSEBE IN SUROVO RAVNANJE': 1260, 'NEVESTNO DELO V SLUŽBI': 119, 'KRŠITEV NEDOTAKLJIVOSTI STANOVANJA': 381, 'DAVÈNA ZATAJITEV': 579, 'PONEVERBA IN NEUPRAVIÈENA UPORABA TUJEGA PREMOŽENJA': 3746, 'ZLORABA URADNEGA POLOŽAJA ALI URADNIH PRAVIC': 347, 'OVERITEV LAŽNE VSEBINE': 899, 'OŠKODOVANJE UPNIKOV': 82, 'KRŠITEV TEMELJNIH PRAVIC DELAVCEV': 2307, 'PREPOVEDANO PREHAJANJE MEJE ALI OZEMLJA DRŽAVE': 1114, 'ZLORABA OSEBNIH PODATKOV': 167, 'VELIKA TATVINA': 11434, 'TATVINA': 23025, 'OŠKODOVANJE TUJIH PRAVIC': 98, 'NEUPRAVIÈENA 

In [102]:
for kraj, slovar in kraj_dejanje.items():
    tmp = sorted([(k, v) for k, v in slovar.items()], key=lambda x: x[1])[::-1]
    print("%s:\n\n" % kraj)
    for x in tmp:
        print("%s  %s" %(kraj,x))

PU KRANJ  ('TATVINA', 23025)
PU KRANJ  ('VELIKA TATVINA', 11434)
PU KRANJ  ('NEPLAÈEVANJE PREŽIVNINE', 6576)
PU KRANJ  ('GOLJUFIJA', 6483)
PU KRANJ  ('POSLOVNA GOLJUFIJA', 5371)
PU KRANJ  ('POŠKODOVANJE TUJE STVARI', 4653)
PU KRANJ  ('PONEVERBA IN NEUPRAVIÈENA UPORABA TUJEGA PREMOŽENJA', 3746)
PU KRANJ  ('GROŽNJA', 2774)
PU KRANJ  ('LAHKA TELESNA POŠKODBA', 2758)
PU KRANJ  ('ZATAJITEV', 2345)
PU KRANJ  ('KRŠITEV TEMELJNIH PRAVIC DELAVCEV', 2307)
PU KRANJ  ('PONAREJANJE LISTIN', 1884)
PU KRANJ  ('NEUPRAVIÈENA PROIZVODNJA IN PROMET S PREPOVEDANIMI DROGAMI, NEDOVOLJENIMI SNOVMI V ŠPORTU IN PREDHOD', 1837)
PU KRANJ  ('NASILJE V DRUŽINI', 1719)
PU KRANJ  ('ZLORABA POLOŽAJA ALI ZAUPANJA PRI GOSPODARSKI DEJAVNOSTI', 1348)
PU KRANJ  ('PONAREJANJE DENARJA', 1333)
PU KRANJ  ('ZANEMARJANJE MLADOLETNE OSEBE IN SUROVO RAVNANJE', 1260)
PU KRANJ  ('PREPOVEDANO PREHAJANJE MEJE ALI OZEMLJA DRŽAVE', 1114)
PU KRANJ  ('PONAREDITEV ALI UNIÈENJE POSLOVNIH LISTIN', 991)
PU KRANJ  ('ODVZEM MLADOLETNE OSEBE', 